In [2]:
import qiskit

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid, Hexagonal13, Hexagonal16, FullyConnected, SquareOctagonal, Hexagonal
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal import jigsaw

from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.benchmarks.bv import bv_circuit
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from PatchedMeasCal import non_standard_coupling_maps


from functools import partial

import qiskit.tools.jupyter

In [21]:
#range_qubits = 6, 17
n_shots = 8000
n_rounds = 5

qubit_range = (11, 17)

circuit_res = []
aim_res = []
sim_res = []
full_res = []
lin_res = []
tpf_res = []
tpf_e_res = []
jr_res = []

backend_generator = FullyConnected

circuit_generators = [state_prep_circuits.plus_state_prep]

pb = Progressbar(20, 5 * n_rounds * len(list(range(*qubit_range))), 'Sims')

for n_qubits in range(*qubit_range):
    
    circuit_round = []
    #aim_round = []
    #sim_round = []
    tpf_round = []
    tpf_e_round = []
    jr_round = []

    for r_num in range(n_rounds):
            
        backend = FullyConnected(n_qubits)
        
        
        err_cmap = non_standard_coupling_maps.error_coupling_map(backend=backend)
        
        # Build tensor patches
#         tpf = TensorPatchFitter(backend, n_shots=n_shots / 2)
#         tpf.build()
        
        tpf_e = TensorPatchFitter(backend, n_shots=n_shots / 2, coupling_map=err_cmap)
        tpf_e.build()
        
#         full = qiskit_full(backend, n_qubits, n_shots)
#         linear = qiskit_linear(backend, n_qubits, n_shots)

        # Run Circuits
        
        circuit = state_prep_circuits.plus_state_prep(backend)      
        pb.tick('Bare: {}'.format(n_qubits))
        # Transpile and run circuit
        tc = qiskit.transpile(circuit,
                              backend,
                              optimization_level=0,
                              initial_layout=list(range(n_qubits))
                             )
        cr = qiskit.execute(circuit, 
                             backend, 
                             shots=n_shots / 2, 
                             optimization_level=0,
                             initial_layout=list(range(n_qubits))
                            ).result().get_counts()
        circuit_round.append(state_prep_circuits.plus_state_dist(cr))

#         # Apply AIM to circuit
#         pb.tick('AIM {}'.format(n_qubits))
#         ar = aim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)
#         aim_round.append(state_prep_circuits.plus_state_dist(ar))

#         # Apply SIM to circuit
#         pb.tick('SIM {}'.format(n_qubits))
#         sr = sim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True) 
#         sr = renormalise_measurement_results(sr, 1)
#         sim_round.append(state_prep_circuits.plus_state_dist(sr))

#         # Apply Full
#         pb.tick('Full {}'.format(n_qubits))
#         fr = full.apply(cr)
#         full_round.append(state_prep_circuits.plus_state_dist(fr))

#         # Apply Linear
#         pb.tick('Lin {}'.format(n_qubits))
#         lr = linear.apply(cr)
#         linear_round.append(state_prep_circuits.plus_state_dist(lr))

#         # Apply CMC to circuit
#         pb.tick('CMC {}'.format(n_qubits))
#         tr = tpf.apply(cr)
#         tpf_round.append(state_prep_circuits.plus_state_dist(tr))
        
        # Apply CMC to circuit
        pb.tick('CMC E{}'.format(n_qubits))
        tr = tpf_e.apply(cr)
        tpf_e_round.append(state_prep_circuits.plus_state_dist(tr))

#         # Jigsaw
#         pb.tick('JIG {}'.format(n_qubits))
#         jr = jigsaw.jigsaw(circuit, backend, n_shots, equal_shot_distribution=True)
#         jr_round.append(state_prep_circuits.plus_state_dist(jr))

    circuit_res.append(circuit_round)
#     aim_res.append(aim_round)
#     sim_res.append(sim_round)
#         full_res.append(full_round)
#         lin_res.append(linear_round)
    #tpf_res.append(tpf_round)
    tpf_e_res.append(tpf_e_round)
    #jr_res.append(jr_round)

results = {
'bare':circuit_res,
'aim':aim_res,
'sim':sim_res,
'full':full_res,
'lin':lin_res,
'cmc':tpf_res,
'cmc_err':tpf_e_res,
'jig':jr_res
}

Sims : [=======>            ] 39.3% CMC E16 ETA: 2191s

In [18]:
results = {
'bare':circuit_res,
'aim':aim_res,
'sim':sim_res,
'full':full_res,
'lin':lin_res,
'cmc':tpf_res,
'cmc_err':tpf_e_res,
'jig':jr_res
}

In [22]:
results

{'bare': [[0.515, 0.5145, 0.4315, 0.4825, 0.46375],
  [0.5057499999999999, 0.49925, 0.5185, 0.5, 0.51075],
  [0.5097499999999999, 0.5567500000000001, 0.5625, 0.549, 0.5575],
  [0.623, 0.5685, 0.51875, 0.5687500000000001, 0.53925],
  [0.5489999999999999, 0.5914999999999999, 0.621, 0.567, 0.604],
  [0.595, 0.67425, 0.643, 0.64175, 0.58025]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [],
 'cmc_err': [[0.21352179713217062,
   0.2065665081874618,
   0.1645274477373126,
   0.20214343265796136,
   0.18599372815426257],
  [0.2332338561952928,
   0.26374318840280714,
   0.24131614144966446,
   0.27705984090735436,
   0.24780655558439907],
  [0.23609338603941193,
   0.25887735682265195,
   0.3499411616001075,
   0.2924489386200255,
   0.2904758279061839],
  [0.3197687303609923,
   0.288018017537626,
   0.22740107840455814,
   0.2917628787006272,
   0.2802966764604862],
  [0.2765597265349782,
   0.3216728629319079,
   0.31090939852635846,
   0.31731525549901524,
   0.33680144017240

In [19]:
results

{'bare': [[0.23249999999999998,
   0.26375000000000004,
   0.263,
   0.24,
   0.23775000000000002]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.1827362740989978,
   0.06991909976914112,
   0.1597147048287414,
   0.08104766357566756,
   0.1009588602194389]],
 'cmc_err': [[0.05910190963009304,
   0.05313479051277026,
   0.09301126549465488,
   0.062468322299207346,
   0.06057898222639191]],
 'jig': [[0.1461388397080226,
   0.17538390178405328,
   0.21344790159818328,
   0.1853847225304397,
   0.16301195071784963]]}

In [16]:
results

{'bare': [[0.23249999999999998,
   0.26375000000000004,
   0.263,
   0.24,
   0.23775000000000002]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.1827362740989978,
   0.06991909976914112,
   0.1597147048287414,
   0.08104766357566756,
   0.1009588602194389]],
 'jig': [[0.1461388397080226,
   0.17538390178405328,
   0.21344790159818328,
   0.1853847225304397,
   0.16301195071784963]]}

In [11]:
results

{'bare': [[0.3245, 0.3125, 0.294, 0.3175, 0.29325],
  [0.35300000000000004, 0.34175, 0.34525, 0.35300000000000004, 0.336],
  [0.40650000000000003, 0.40525, 0.36725, 0.39275, 0.40900000000000003],
  [0.435, 0.37875000000000003, 0.40800000000000003, 0.395, 0.41725],
  [0.41675, 0.47125, 0.4245, 0.44025, 0.47100000000000003],
  [0.45325000000000004, 0.49425, 0.49199999999999994, 0.4615, 0.46625],
  [0.534, 0.5185, 0.53725, 0.5405, 0.53575],
  [0.5589999999999999, 0.552, 0.5247499999999999, 0.573, 0.5365],
  [0.58775, 0.53075, 0.579, 0.531, 0.5650000000000001],
  [0.57875, 0.6165, 0.5902499999999999, 0.613, 0.57775],
  [0.6465000000000001,
   0.5840000000000001,
   0.62775,
   0.6214999999999999,
   0.63525]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.3355552978612375,
   0.10055643173501949,
   0.06019205615871892,
   0.378307434726201,
   0.23400802233726453],
  [0.3140342958954466,
   0.10322159691812832,
   0.1163247161870079,
   0.09862679046977485,
   0.11883398259

In [5]:
results

{'bare': [[0.3245, 0.3125, 0.294, 0.3175, 0.29325],
  [0.35300000000000004, 0.34175, 0.34525, 0.35300000000000004, 0.336],
  [0.40650000000000003, 0.40525, 0.36725, 0.39275, 0.40900000000000003],
  [0.435, 0.37875000000000003, 0.40800000000000003, 0.395, 0.41725],
  [0.41675, 0.47125, 0.4245, 0.44025, 0.47100000000000003],
  [0.45325000000000004, 0.49425, 0.49199999999999994, 0.4615, 0.46625],
  [0.534, 0.5185, 0.53725, 0.5405, 0.53575],
  [0.5589999999999999, 0.552, 0.5247499999999999, 0.573, 0.5365],
  [0.58775, 0.53075, 0.579, 0.531, 0.5650000000000001],
  [0.57875, 0.6165, 0.5902499999999999, 0.613, 0.57775],
  [0.6465000000000001,
   0.5840000000000001,
   0.62775,
   0.6214999999999999,
   0.63525]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.3355552978612375,
   0.10055643173501949,
   0.06019205615871892,
   0.378307434726201,
   0.23400802233726453],
  [0.3140342958954466,
   0.10322159691812832,
   0.1163247161870079,
   0.09862679046977485,
   0.11883398259

In [25]:
results

{'bare': [[0.28025, 0.34225, 0.3065, 0.30825, 0.26575],
  [0.35350000000000004, 0.374, 0.36025, 0.34525, 0.36324999999999996],
  [0.381, 0.38375, 0.39325, 0.383, 0.3865],
  [0.40075,
   0.43575,
   0.40349999999999997,
   0.39849999999999997,
   0.45325000000000004],
  [0.46075, 0.40750000000000003, 0.44975, 0.45449999999999996, 0.45225],
  [0.455, 0.45125, 0.5065, 0.45675, 0.50875],
  [0.5105000000000001, 0.51025, 0.504, 0.54775, 0.505],
  [0.53, 0.50825, 0.56525, 0.5345, 0.50225],
  [0.55175, 0.5705, 0.565, 0.5175000000000001, 0.60775],
  [0.60625, 0.5872499999999999, 0.6174999999999999, 0.6135, 0.6025],
  [0.5972500000000001, 0.6435, 0.6587500000000001, 0.64025, 0.613]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.18763105971810023,
   0.12112439929353602,
   0.3240318159234237,
   0.15460440959708527,
   0.16649727260555858],
  [0.1477494734886356,
   0.270292349779756,
   0.11519895518674922,
   0.10631119028766345,
   0.2647008599814858],
  [0.36379795022619144,


[[0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544],
 [0.1258825408085893,
  0.05456219094257392,
  0.07325000322209063,
  0.0966495634587734,
  0.1677002952465544]]

In [10]:
range_results_5_9 = {'bare': [
  [0.25425,
   0.27899999999999997,
   0.22950000000000004,
   0.23124999999999996,
   0.22899999999999998],
  [0.28125, 0.27925, 0.33325, 0.34475, 0.2995],
  [0.34325000000000006, 0.35650000000000004, 0.34575, 0.34725, 0.33275],
  [0.3395, 0.40275, 0.40349999999999997, 0.381, 0.3745],
  [0.4265, 0.43225, 0.4435, 0.41375, 0.4205]],
 'aim': [[0.25933333333333336,
   0.2845,
   0.23783333333333334,
   0.22900000000000004,
   0.2418333333333333],
  [0.2823333333333333,
   0.29783333333333334,
   0.3431666666666667,
   0.3378333333333334,
   0.30266666666666664],
  [0.34983333333333333,
   0.3393333333333333,
   0.35333333333333333,
   0.3421666666666667,
   0.3461666666666667],
  [0.3308333333333333,
   0.39666666666666667,
   0.4028333333333334,
   0.38016666666666665,
   0.3703333333333333],
  [0.4145,
   0.43083333333333335,
   0.43883333333333335,
   0.4068333333333333,
   0.41833333333333333]],
 'sim': [[0.2615000006399932,
   0.27899999827635485,
   0.22700000185624225,
   0.23737500573863496,
   0.24399999852292248],
  [0.2742500024376749,
   0.29375000573545534,
   0.3387500022264612,
   0.3317500018980645,
   0.30012499220318983],
  [0.35400000104328494,
   0.3312500031583112,
   0.3522500033399009,
   0.3443750034218009,
   0.34424999890510305],
  [0.3336249934940464,
   0.4151249909651875,
   0.40475000864400107,
   0.3863750041331712,
   0.36774999475335457],
  [0.4268749944110371,
   0.4326249952823894,
   0.43737500309073024,
   0.41449999637273155,
   0.4226249989598382]],
 'full': [[0.028169244047607733,
   0.03566008623929218,
   0.024241716783764633,
   0.04010192174173138,
   0.030613049183080276],
  [0.04959782847669303,
   0.028530788871486468,
   0.04364105670787061,
   0.059993989079032084,
   0.05263049296473882],
  [0.040562746928451354,
   0.08014222720037073,
   0.031403797352688834,
   0.06954693741229306,
   0.06276843424502176],
  [0.075155600619566,
   0.055823701575610896,
   0.05128545934877238,
   0.05845428252709778,
   0.08323735368975849],
  [0.08148669115439477,
   0.09220154052262342,
   0.07764099987198986,
   0.07280963161585635,
   0.08252148684934174]],
 'lin': [[0.024231969667889874,
   0.04095388810436151,
   0.03007665205072968,
   0.04216172788513345,
   0.027771422412388225],
  [0.04514167148145465,
   0.033287412069318234,
   0.03662967457960453,
   0.04996276913800618,
   0.045559330999241476],
  [0.05105993900657119,
   0.08334992418708992,
   0.0515818728159837,
   0.0566692731175768,
   0.05559186365693375],
  [0.07140723099969742,
   0.05678227708517353,
   0.06287888028895067,
   0.056044188904436854,
   0.07920955968529902],
  [0.07721663405107437,
   0.08080556467626937,
   0.08003173135263963,
   0.0698723147867481,
   0.07827155391953816]],
 'cmc': [[0.12201236007987659,
   0.11599322081417707,
   0.06521293442017612,
   0.06847350005568625,
   0.21528851997022785],
  [0.2021569448611581,
   0.08241855024225725,
   0.06168334923104124,
   0.137105599789151,
   0.30945404553648953],
  [0.10170647640139618,
   0.11969093775401651,
   0.4065321378852029,
   0.12642460107632436,
   0.4730805190769869],
  [0.30978505758263203,
   0.488421281414842,
   0.5056139141257386,
   0.5608138679918931,
   0.4258176482368235],
  [0.19815503358022407,
   0.17637361209234753,
   0.20731051473619533,
   0.3274144230005122,
   0.287033270771123]],
 'cmc_err':[[0.05910190963009304,
   0.05313479051277026,
   0.09301126549465488,
   0.062468322299207346,
   0.06057898222639191],
  [0.09838371818786018,
   0.09320029683863024,
   0.06822319930427334,
   0.07848089550630982,
   0.08973776998724797],
  [0.15305376574535395,
   0.08085778160256052,
   0.10269345143245706,
   0.09874145751836716,
   0.1295281568560861],
  [0.14587476526352045,
   0.16455319822941572,
   0.16002651027927617,
   0.10933331749677022,
   0.14058769754873118],
  [0.13547329415281012,
   0.13888312720636786,
   0.1390332847916631,
   0.10914422071866908,
   0.1467798315763022]],
 'jig': [[0.1926026634784389,
   0.1934622029889952,
   0.1403544840894954,
   0.1397448453559258,
   0.19521596964629412],
  [0.19520759546342575,
   0.22510056715456156,
   0.2245447359356595,
   0.25253822094887873,
   0.17567312203647156],
  [0.2587508695689904,
   0.2521848721579589,
   0.25650302672793884,
   0.28427459024022,
   0.24923181542221046],
  [0.2223816501219525,
   0.2953776637228855,
   0.2677525690917426,
   0.32235035240853627,
   0.2211292577827314],
  [0.3405984038533609,
   0.30633227083253456,
   0.3350576935442423,
   0.3137087266844817,
   0.31646097150498004]]}

In [24]:
range_results_10_13 = {'bare': [[0.4375, 0.43224999999999997, 0.43025, 0.46475, 0.43425],
  [0.47325, 0.47025, 0.4915, 0.467, 0.48975],
  [0.5155000000000001, 0.53925, 0.5415, 0.49124999999999996, 0.5175],
  [0.55675, 0.57375, 0.54225, 0.5694999999999999, 0.5625]],
 'aim': [[0.45049999999999996,
   0.4411666666666667,
   0.4221666666666667,
   0.44916666666666666,
   0.4475],
  [0.49616666666666664,
   0.489,
   0.5021666666666667,
   0.45966666666666667,
   0.4935],
  [0.512,
   0.5313333333333333,
   0.5541666666666667,
   0.4881666666666667,
   0.5108333333333334],
  [0.5569999999999999,
   0.5660000000000001,
   0.5448333333333333,
   0.5773333333333334,
   0.554]],
 'sim': [[0.4354999958812552,
   0.4323750053368347,
   0.43862500235216967,
   0.4477499984935494,
   0.4377500037910866],
  [0.49287498523118767,
   0.4912500005004767,
   0.5031249955681689,
   0.4626249949759497,
   0.48287499007127793],
  [0.5202499974648891,
   0.5507500047665015,
   0.5434999987102275,
   0.4973749952511901,
   0.5164999931051428],
  [0.5571249952285019,
   0.5670000037804128,
   0.5394999949743363,
   0.580874999439966,
   0.5536250044453535]],
 'full': [],
 'lin': [],
 'cmc': [[0.18625559417669113,
   0.5159813657804004,
   0.26776622133167716,
   0.1794499975848982,
   0.21651814493257965],
  [0.2727447314919319,
   0.24941680340397349,
   0.2946000176491917,
   0.4482575863259608,
   0.430934674354984],
  [0.3029683745358932,
   0.25442061793701637,
   0.38809863393103017,
   0.4585660962984617,
   0.30460761211551207],
  [0.27933672364148615,
   0.5508862326769172,
   0.6876242993657022,
   0.5130934504202792,
   0.43243424616591547]],
 'cmc_err':[[0.15012727250364272,
   0.15066714721859653,
   0.1737260505102282,
   0.19868600098396655,
   0.19394075908801595],
  [0.21352179713217062,
   0.2065665081874618,
   0.1645274477373126,
   0.20214343265796136,
   0.18599372815426257],
  [0.2332338561952928,
   0.26374318840280714,
   0.24131614144966446,
   0.27705984090735436,
   0.24780655558439907],
  [0.23609338603941193,
   0.25887735682265195,
   0.3499411616001075,
   0.2924489386200255,
   0.2904758279061839]],
 'jig': [[0.29924381243896736,
   0.3428042722577071,
   0.3241156015205776,
   0.3367886939968197,
   0.3473505672543244],
  [0.3208269630169426,
   0.3692246209106695,
   0.3635871595538913,
   0.34009447559743644,
   0.37665641392171995],
  [0.4035115383814037,
   0.37923540276748824,
   0.41468716804687344,
   0.3485309312826105,
   0.3825784178309302],
  [0.43300421549695317,
   0.46050304811032106,
   0.4442183303141965,
   0.4310157858841982,
   0.40288976658968917]]}

In [25]:
  range_results_14_16 = {'bare': [[0.565, 0.5925, 0.54, 0.58775, 0.5760000000000001],
  [0.6535,
   0.6047499999999999,
   0.5607500000000001,
   0.5882499999999999,
   0.58075],
  [0.60375, 0.634, 0.56175, 0.59425, 0.6085]],
 'aim': [[0.5718333333333333,
   0.5915,
   0.5466666666666666,
   0.5820000000000001,
   0.5655],
  [0.646, 0.6053333333333333, 0.5718333333333333, 0.597, 0.5826666666666667],
  [0.611, 0.6313333333333333, 0.5728333333333333, 0.5954999999999999, 0.624]],
 'sim': [[0.5762499955845308,
   0.5921250015575097,
   0.5466250046000567,
   0.587625000367987,
   0.5722500000003456],
  [0.6497499972818807,
   0.6123749974130025,
   0.5798749985064224,
   0.6101250053445908,
   0.5751249993480305],
  [0.6083749919070129,
   0.6324999956368995,
   0.5602500027068816,
   0.5998749947184099,
   0.6141249904350121]],
 'full': [],
 'lin': [],
 'cmc': [[0.6676548974662906,
   0.6872630820666069,
   0.27569991708851443,
   0.6857457488824181,
   0.3616853140300104],
  [0.3793911187163348,
   0.30996019371210715,
   0.7618342499956952,
   0.5030328676732263,
   0.6451928428046132],
  [0.33635591283103805,
   0.6904769081289124,
   0.2991978506489069,
   0.3070314798313424,
   0.4399353015097175]],
 'cmc_err':[[0.3197687303609923,
   0.288018017537626,
   0.22740107840455814,
   0.2917628787006272,
   0.2802966764604862],
  [0.2765597265349782,
   0.3216728629319079,
   0.31090939852635846,
   0.31731525549901524,
   0.3368014401724098],
  [0.345029959330152,
   0.3796904700356879,
   0.355832135357939,
   0.411376666122396,
   0.3326025804155448]],
 'jig': [[0.4502622948511955,
   0.46486857238675816,
   0.4309150358562449,
   0.4201669650232095,
   0.4615905967129216],
  [0.5120870583674857,
   0.496261561070934,
   0.4328517723306654,
   0.48481629995481945,
   0.49279642824053815],
  [0.4125952584574052,
   0.48893227495606073,
   0.4700284841419163,
   0.4581651659321542,
   0.44411254238876335]]}
    


In [21]:
range_results_cmc_conv = {'bare': [[0.22800000000000004,
   0.276,
   0.22649999999999998,
   0.24625000000000002,
   0.26599999999999996],
  [0.30375, 0.33675, 0.30875, 0.332, 0.33225],
  [0.38175, 0.33425000000000005, 0.35, 0.30975, 0.36974999999999997],
  [0.39299999999999996,
   0.35374999999999995,
   0.402,
   0.3705,
   0.37825000000000003],
  [0.43, 0.41325, 0.42075, 0.45075, 0.39475],
  [0.5117499999999999,
   0.432,
   0.42400000000000004,
   0.4745,
   0.43674999999999997],
  [0.46575, 0.495, 0.49074999999999996, 0.462, 0.46275],
  [0.522, 0.50025, 0.5015000000000001, 0.49725, 0.49424999999999997],
  [0.5395, 0.5395000000000001, 0.5377500000000001, 0.52525, 0.533],
  [0.55375, 0.5547500000000001, 0.5645, 0.5660000000000001, 0.56525],
  [0.5920000000000001, 0.58375, 0.5974999999999999, 0.59425, 0.60375],
  [0.6252500000000001, 0.61825, 0.6165, 0.6155, 0.64725]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.05358225009386208,
   0.25429361389550936,
   0.34021763745659556,
   0.2797774360388859,
   0.07673402877032487],
  [0.13942624509020352,
   0.09993643325706647,
   0.11786690295919888,
   0.10210052392676389,
   0.1406134746515798],
  [0.13121877653455594,
   0.11152721435917418,
   0.40687007793114405,
   0.20262232205279346,
   0.18227712564542703],
  [0.141244595797826,
   0.255235047736292,
   0.14693579082143643,
   0.4809055461030687,
   0.3483854682269039],
  [0.24238155859315236,
   0.13971478251212488,
   0.5104084374122126,
   0.22947809129985652,
   0.1573919958168593],
  [0.5523038931435346,
   0.15988157359831212,
   0.13433522122397562,
   0.275967438268855,
   0.17556336854847826],
  [0.18835336135236502,
   0.5944753899982071,
   0.2391338303376886,
   0.34968506806933775,
   0.64981874671702],
  [0.5682083045398937,
   0.23352151283491407,
   0.22342151718396336,
   0.23424771719885906,
   0.5698882454635708],
  [0.5313459556961398,
   0.6706903282408925,
   0.43101168257193534,
   0.29227203794641776,
   0.67571299762956],
  [0.26349422514953774,
   0.6788406971085215,
   0.47946769009227286,
   0.5376713904124062,
   0.29524281710190636],
  [0.39745242069263864,
   0.6752266866444963,
   0.38163844599658686,
   0.7169692360266309,
   0.8031060320089987],
  [0.36308213219985025,
   0.6071966270170672,
   0.36300667349218035,
   0.3661786920732499,
   0.5935334400211769]],
 'cmc_err': [[0.19137277358387011,
   0.18054076426783944,
   0.24201355553881643,
   0.19579756783840557,
   0.2107156953478425],
  [0.2492366844679092,
   0.19955066217549522,
   0.2647407257870535,
   0.21196611480150757,
   0.2765471672039492],
  [0.2823504613176978,
   0.23381537718833068,
   0.22877559481773774,
   0.30228434247394975,
   0.2595991842670315],
  [0.2850973653959887,
   0.2209218550850902,
   0.29344616927380446,
   0.26836998927540723,
   0.30434209701882364],
  [0.3221223124030619,
   0.34541775925059637,
   0.30183299226114624,
   0.33383922155418305,
   0.33180323146081153],
  [0.42405396409497037,
   0.3195683439773323,
   0.33158158716920344,
   0.32571881868584446,
   0.32728424757579344]],
 'jig': []}


 [[0.22894792046094908,
   0.22449899624036923,
   0.1946106489183559,
   0.21606778903015822,
   0.20467685047408946],
  [0.24454438970902237,
   0.2611726393726255,
   0.20097926112312087,
   0.2481172242998833,
   0.23770519130172657],
  [0.3021870231542739,
   0.27848842090486287,
   0.23435070071445402,
   0.2573169152256401,
   0.27039291471254884],
  [0.33187619978682464,
   0.23736768334433178,
   0.27459491811870906,
   0.24782230280276274,
   0.2706192312151767],
  [0.2920619518965532,
   0.32435213101195415,
   0.2970498552611606,
   0.3403652679957535,
   0.3359538224715669],
  [0.30419063484080966,
   0.4003533626726513,
   0.3811767390949459,
   0.340866903243587,
   0.33519386102616405],
  [0.38600545621738613,
   0.393960478057646,
   0.3695828676588948,
   0.4138376649256517,
   0.3792580187337602],
  [0.43581738206510073,
   0.44532137384847365,
   0.40158142932020335,
   0.43643645799181985,
   0.4117909003583926],
  [0.3844052019977697,
   0.42876589864880094,
   0.4441923728568264,
   0.4292093274454847,
   0.3722070138012409],
  [0.44777692716966555,
   0.43787841880997874,
   0.4643059844018377,
   0.47054602251218225,
   0.4195307956968793],
  [0.4829104232721044,
   0.4446343175698168,
   0.4820425092172808,
   0.5116104756224322,
   0.43358170376629634]]

In [26]:
range_results_cmc_2 = {'bare': [[0.28025, 0.34225, 0.3065, 0.30825, 0.26575],
  [0.35350000000000004, 0.374, 0.36025, 0.34525, 0.36324999999999996],
  [0.381, 0.38375, 0.39325, 0.383, 0.3865],
  [0.40075,
   0.43575,
   0.40349999999999997,
   0.39849999999999997,
   0.45325000000000004],
  [0.46075, 0.40750000000000003, 0.44975, 0.45449999999999996, 0.45225],
  [0.455, 0.45125, 0.5065, 0.45675, 0.50875],
  [0.5105000000000001, 0.51025, 0.504, 0.54775, 0.505],
  [0.53, 0.50825, 0.56525, 0.5345, 0.50225],
  [0.55175, 0.5705, 0.565, 0.5175000000000001, 0.60775],
  [0.60625, 0.5872499999999999, 0.6174999999999999, 0.6135, 0.6025],
  [0.5972500000000001, 0.6435, 0.6587500000000001, 0.64025, 0.613]],
 'aim': [],
 'sim': [],
 'full': [],
 'lin': [],
 'cmc': [[0.18763105971810023,
   0.12112439929353602,
   0.3240318159234237,
   0.15460440959708527,
   0.16649727260555858],
  [0.1477494734886356,
   0.270292349779756,
   0.11519895518674922,
   0.10631119028766345,
   0.2647008599814858],
  [0.36379795022619144,
   0.386838965271883,
   0.351206183334842,
   0.2902114664361831,
   0.6513463405975657],
  [0.23840574420698873,
   0.18812375191299535,
   0.1984002423542729,
   0.1386609168737012,
   0.5798638083608105],
  [0.18039660691444376,
   0.7580900958733008,
   0.35357860752469134,
   0.17698938332423336,
   0.3932398226223104],
  [0.7280257389316738,
   0.746398797769257,
   0.7398654121309339,
   0.19260973960215672,
   0.36004072045732155],
  [0.5490760716600565,
   0.44214936560757734,
   0.22142876954071744,
   0.6740539855379124,
   0.24229254858068278],
  [0.547753484476457,
   0.6992701038854923,
   0.562822633279497,
   0.6712412777258755,
   0.6996210695712825],
  [0.2726694200515931,
   0.2884052603978325,
   0.28721112970903434,
   0.4534390290690393,
   0.2946438253479356],
  [0.4440217939387644,
   0.742759884417757,
   0.7660011487198338,
   0.758941467171486,
   0.6596463964351803],
  [0.3303704632299451,
   0.4875838582557468,
   0.3719403449681935,
   0.37042841982834873,
   0.33703068722636054]],
 'jig': []}